# lesson-3

### Ответы на вопросы

**3 товара с акцией** (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)

- Можно просто взять первые 3 товара с акцией из полного output ALS
- Можно обучать алгоритм только на товарах со скидкой, которая есть именно сейчас
- В user-item матрице будет стоять информация о пукупке товара с той скидкой, которая была на момент покупки. Её можно учитывать ставя в user-item матрицу процентную стоимость товара со скидкой, например если скидка на товар была 10%, то вес в user-item матрице будет 0.9, так как чем больше скидка, тем меньше пользователя заинтересовал сам товар, а не его дешевизна. Например, если скидка 100%, тоесть товар отдаётся бесплатно, то его может взять кто угодно, но это не значит, что пользователю инетесен сам товар. Поэтому значение в матрице будет 0, как будто он и не покупал этот товар.)

**1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?) 

- Можно просто взять первый товар который еще не покупал пользователь из полного output ALS
- Можно обучать алгоритм только на товарах которые еще не покупал пользователь. Но тогда придётся обучать алгоритм занового под каждого пользователя
- Можно предварительно отфильтровать самые популярные товары, так как их скорее всего большинство пользователей уже покупало

**1 товар для роста среднего чека** (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)

- Нужно найти сумму всех товаров в одной корзине (basket_id) и разделить её на количество товаров в этой корзине, это будет средняя стоимость товара в корзине. Найти все корзины у пользователя, найти в них среднюю стоимость товара, сложить их всех и разделить на количество найденных коризн у пользователя. Это будет средняя стоимость товара у пользователя. Теперь будем считать товар подходящим, если он дороже средней стоимости товара у пользователя хотя бы на один цент, так как если пользователи будут покупать такие товары, то их средний чек должен вырасти, а на сколько вырости, нам не указано. 
- Можно просто взять первый товар, который дороже средней стоимости товара у пользователя из полного output ALS
- Можно обучать алгоритм только на товарах, которые дороже средней стоимости товара у пользователя. Но тогда придётся обучать алгоритм занового под каждого пользователя.
- Можно предварительно отфильтровать самые дешёвые товары, так как их стоимость наврятли будет больше средней стоимости товара у большинства пользователей

### Подбор гиперпараметров ALS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

In [2]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [3]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [4]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [6]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [7]:
# Заведем фиктивный item_id

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

D:\anakonda\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

### ALS

In [9]:
parameters = {'factors':[100, 200, 300], 'regularization':[0.0005, 0.001, 0.002], 'iterations':[10, 15, 30],
             'calculate_training_loss':[True], 'num_threads':[4], 'random_state':[42]}

In [10]:
def grid_search(X, y, class_model, parameters):
    
    best_params = {}
    best_score = 0
    
    for params in itertools.product(*parameters.values()):
        model = class_model(**dict(zip(parameters.keys(), params)))
        model.fit(X, show_progress=False)
        y['res'] = y['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
        score = y.apply(lambda row: precision_at_k(row['res'], row['actual']), axis=1).mean()
        if score > best_score:
            best_score = score
            best_params = dict(zip(parameters.keys(), params))
        
    print(best_score)             
    return best_params

#### Без взвешивания

In [11]:
%%time

als_params = grid_search(csr_matrix(user_item_matrix).T.tocsr(), 
                       result[['user_id', 'actual']], AlternatingLeastSquares, parameters)

0.16199804113613891
Wall time: 59min 3s


In [25]:
als_params

{'factors': 100,
 'regularization': 0.0005,
 'iterations': 15,
 'calculate_training_loss': True,
 'num_threads': 4,
 'random_state': 42}

In [26]:
%%time

model = AlternatingLeastSquares(**als_params)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 56 s


0.16199804113613891

#### TF-IDF взвешивание

In [27]:
tfidf_user_item_matrix = tfidf_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [28]:
%%time

als_tfidf_params = grid_search(csr_matrix(tfidf_user_item_matrix).T.tocsr(), 
                             result[['user_id', 'actual']], AlternatingLeastSquares, parameters)

0.16474045053868555
Wall time: 58min 1s


In [29]:
als_tfidf_params

{'factors': 100,
 'regularization': 0.001,
 'iterations': 15,
 'calculate_training_loss': True,
 'num_threads': 4,
 'random_state': 42}

In [45]:
%%time

model = AlternatingLeastSquares(**als_tfidf_params)

model.fit(csr_matrix(tfidf_user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_tfidf'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_tfidf'], row['actual']), axis=1).mean()

  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 55.3 s


0.16474045053868555

#### BM25 взвешивание

In [31]:
bm25_user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [32]:
%%time

als_bm25_params = grid_search(csr_matrix(bm25_user_item_matrix).T.tocsr(), 
                               result[['user_id', 'actual']], AlternatingLeastSquares, parameters)

C:\Users\D9F8~1\AppData\Local\Temp/ipykernel_9784/330473836.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['res'] = y['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))


0.20205680705190693
Wall time: 57min 58s


In [33]:
als_bm25_params

{'factors': 200,
 'regularization': 0.002,
 'iterations': 10,
 'calculate_training_loss': True,
 'num_threads': 4,
 'random_state': 42}

In [46]:
%%time

model = AlternatingLeastSquares(**als_bm25_params)

model.fit(csr_matrix(bm25_user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()

  0%|          | 0/10 [00:00<?, ?it/s]

Wall time: 1min 54s


0.20205680705190693

#### Сохраним результаты

In [47]:
result.head(3)

,user_id,actual,als,als_tfidf,als_bm25
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1105488, 1033142, 5569374, 979707, 1005186]","[1041796, 999999, 995242, 965766, 1082185]","[995242, 999999, 1082185, 1033142, 965766]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 1133018, 910032, 929668, 5569327]","[951590, 1106523, 1133018, 938700, 999999]","[1133018, 999999, 1106523, 951590, 1053690]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[866211, 1051516, 1007195, 878996, 904360]","[1023720, 1007195, 1051516, 878996, 1127831]","[904360, 1082185, 878996, 866211, 999999]"


In [48]:
result.to_csv('gs_predictions_mf.csv', index=False)  # gs - grid search, mf - matrix factorization

### Реализуем префильтрацию

In [49]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [50]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [51]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [52]:
class Prefilter_Items():
    
    def __init__(self):
        
        self.popular = None
        self.not_popular = None
        self.old = None
        self.low_price = None
        self.high_price = None
        self.drop_department = None
    
    def fit(self, data_train, item_features=None, drop_department=None):
        
        popularity = data_train.groupby('item_id')['user_id'].nunique().reset_index()
        popularity['user_id'] /= data_train['user_id'].nunique()
        popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
        
        self.popular = popularity[popularity['share_unique_users'] > 0.5].item_id.tolist()
        
        self.not_popular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist()
        
        self.old = data_train[data_train['week_no'] <= data_train['week_no'].max() - 52].item_id.tolist()
        
        price = data_train[['item_id', 'sales_value']]
        price['sales_value'] /= (np.maximum(data_train['quantity'], 1))
        price = data_train.groupby('item_id')['sales_value'].last().reset_index()
        
        self.low_price = price[price['sales_value'] < 1].item_id.tolist()
        
        self.high_price = price[price['sales_value'] > 100].item_id.tolist()
        
        if drop_department is not None:
            self.drop_department = item_features.loc[item_features['department'].isin(drop_department)].item_id.tolist()
            
        
    def transform(self, data):
        
        # Уберем самые популярные товары (их и так купят)
        data = data[~data['item_id'].isin(self.popular)]
        
        # Уберем самые НЕ популярные товары (их и так НЕ купят)
        data = data[~data['item_id'].isin(self.not_popular)]
        
        # Уберем товары, которые не продавались за последние 12 месяцев
        data = data[~data['item_id'].isin(self.old)]
        
        # Уберем не интересные для рекоммендаций категории (department)
        if self.drop_department is not None:
            data = data[~data['item_id'].isin(self.drop_department)]
        
        # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
        data = data[~data['item_id'].isin(self.low_price)]
        
        # Уберем слишком дорогие товары
        data = data[~data['item_id'].isin(self.high_price)]
        
        return data
    

In [53]:
data_train.shape, data_test.shape

((2278490, 12), (118314, 12))

In [54]:
prefilter_items = Prefilter_Items()
prefilter_items.fit(data_train, item_features, drop_department=['PASTRY', 'TRAVEL & LEISUR'])
data_train = prefilter_items.transform(data_train)
data_test = prefilter_items.transform(data_test)

C:\Users\D9F8~1\AppData\Local\Temp/ipykernel_9784/17227979.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['sales_value'] /= (np.maximum(data_train['quantity'], 1))


In [55]:
data_train.shape, data_test.shape

((35132, 12), (6382, 12))